## Alternative Process Plan Scheduling

This notebook demonstrates how to model and solve the classical Resource-Constrained Project Scheduling Problem with Alternative Process Plan
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.

### Ideas how to model it 

[Slides from the video](https://icaps17.icaps-conference.org/tutorials/T3-Introduction-to-CP-Optimizer-for-Scheduling.pdf)

Slide 65 -- Structural constraints

**Span constraint - (AND node)**

span(x,[y1,...,yn])

if x is present it spands all rpesent intervals from {y1,...yn}
that is. at least one of yi supports the start (resp. end) of x

if x is absent, then all yi are absent to

**Alternative constraint - (OR node)**

alterantive(x, [y1,...,yn])
if x is present, then **exactly one** of the {y1,..,yn} is present and synchronized with x (sa,e start and end value)

if x is absent then all yi are absent too

alternative(x, [y1,...,yn],k) k:integer expression is x is present then exaclt k of the {y1,...,yn} are present and synchronized with x (same start and end value) if x is absent, then all yi are absent too

combine with presenceOf(task[i]) => presenceOf(task[j])

Sequence constraints

Sequencing constraints are unary constraints on a sequence variable p
- first(p,x) : if x is present it is the first one in the permutation
- last(p,x) : if x is present it is the last one in the permutation
- prev(p,x,y) : if both x and y are present, x appears right before y in the permutation
- before(p,x,y) : if both x and y are present, x appears somewhere before y in the permutation

### Problem Definition

RCPSP-AS is a generalization of RCPSP, which relaxes the assumption that the project has a fixed structure (all tasks in the project must always be scheduled fot the project to be complete). RCPSP-AS allows cetrain tasks to be omited from the schedule, its instance define a set of subgraphs, each which contains multiple alternative branches (alternative process plans).

Let the production consist of $N$ indivisible tasks processed on unary machines according to a process plan; define tasks $i\in[1..N]$ and machines $k\in[1..M]$. Each task $i$ has duration $PT_i>0$ and a fixed machine assignment $h_i\in[1..M]$. Precedence relations together with alternative process plans are specified via a Petri net (a directed bipartite graph with transitions and places). General temporal constraints are given by a matrix $L=(\ell_{ij})$ over the task set, where each $\ell_{ij}\in\mathbb{R}$ enforces $s_i+\ell_{ij}\le s_j$ with $s_i$ the start time of task $i$; the earliest start time $ES_i$ is the minimum time at which task $i$ can be scheduled with respect to these temporal constraints alone (ignoring resources). Assume $\ell_{ij}\ge PT_i$ whenever transition $T_i$ is a direct predecessor of $T_j$ in the Petri net, while other $\ell_{ij}$ values are arbitrary; if there is no temporal constraint from $i$ to $j$, set $\ell_{ij}=-\infty$. Release times and deadlines can be imposed directly as bounds $s_i\ge r_i$ and $s_i\le \tilde d_i$ (or equivalently $\mathrm{startOf}(x_i)\ge r_i$ and $\mathrm{endOf}(x_i)\le d_i$ if deadlines are on completion). For each machine $k$ let $\mathrm{TM}_k=[S^{(k)}_{ij}]$ denote the transition (setup) matrix, so that if task $j$ immediately follows task $i$ on machine $k$ a setup time $S^{(k)}_{ij}$ is incurred; if setups are machine-independent, use $S^{(k)}_{ij}\equiv S_{ij}$ for all $k$; setup times satisfy the triangle inequality $S^{(k)}_{ij}+S^{(k)}_{jk}\ge S^{(k)}_{ik}$. The objective is to minimize the makespan $C_{\max}$. 

This problem can be classified as $PS_M,1,1\mid alt,\ temp,\ S_{ij}\mid C_{\max}$ in the $\alpha\mid\beta\mid\gamma$ notation (with $temp$ for generalized temporal constraints and $S_{ij}$ for sequence-dependent setup times, or $S^{(k)}_{ij}$ when machine-specific), and under $PS_M,1,1$ there are $M$ unary-capacity machines with each task requiring at most one unit of a single machine; since there are no additional renewable resources, the problem can also be viewed as a machine-scheduling problem with sequence-dependent setups.


### CPLEX Formulation

$$
\begin{aligned}
\min\quad
& C_{\max} \;=\; \max_{i \in [1..N]} \mathrm{endOf}(x_i)
& & & \text{(1)} \\[6pt]
\text{s.t.}\quad
& \mathrm{startBeforeStart}(x_i, x_j, \ell_{ij})
& & \forall (i,j)\ \text{with}\ \ell_{ij}>-\infty
& \text{(2)} \\[4pt]
& \mathrm{span}\!\bigl(x_p,\ [\,y_{pi}\,]_{i\in A[p]}\bigr)
& & \forall p \in P_{\text{AND}}
& \text{(3)} \\[4pt]
& \mathrm{alternative}\!\bigl(x_p,\ [\,y_{pi}\,]_{i\in A[p]}\bigr)
& & \forall p \in P_{\text{OR}}
& \text{(4)} \\[4pt]
& \mathrm{noOverlap}\!\bigl(\mathrm{seq}_k,\ \mathrm{TM}_k\bigr)
& & \forall k \in [1..M]
& \text{(5)} \\[6pt]
& \mathrm{presenceOf}(x_i) = \max_{p \mid i \in A[p]} \mathrm{presenceOf}(y_{pi})
& & \forall i \in [1..N]
& \text{(6)} \\[4pt]
& \mathrm{presenceOf}(y_{pi}) \Rightarrow \bigl( \mathrm{startOf}(x_i) = \mathrm{startOf}(y_{pi}) \bigr)
& & \forall p \in P_{\text{AND}} \cup P_{\text{OR}},\ i\in A[p]
& \text{(7)} \\[6pt]
& \text{interval } x_i,\ \text{optional, size} = PT_i
& & \forall i \in [1..N]
& \text{(8a)} \\[4pt]
& \text{interval } y_{pi},\ \text{optional, size} = PT_i
& & \forall p \in P_{\text{AND}} \cup P_{\text{OR}},\ i\in A[p]
& \text{(8b)} \\[4pt]
& \text{interval } x_p,\ \text{optional}
& & \forall p \in P_{\text{AND}} \cup P_{\text{OR}}
& \text{(8c)} \\[4pt]
& \text{sequence } \mathrm{seq}_k\ \text{over }\{\,x_i \mid h_i = k\,\}
& & \forall k \in [1..M]
& \text{(8d)}
\end{aligned}
$$

**Objective:**
- **(1)** Minimize the makespan $C_{\max} = \max_{i\in[1..N]} \mathrm{endOf}(x_i)$.

**Modeling constraints:**
- **(2)** Temporal constraints: enforce general time lags $\ell_{ij}$; for direct precedences, typically $\ell_{ij}\ge PT_i$.
- **(3)** AND composition (span): if $x_p$ is present, it spans all present children $y_{pi}$; if $x_p$ is absent, all $y_{pi}$ are absent.
- **(4)** OR composition (alternative): if $x_p$ is present, exactly one child $y_{pi}$ is selected and synchronized with $x_p$.
- **(5)** Sequence-dependent setups and unary capacity: on each machine $k$, tasks follow a non-overlapping order with setup times from $\mathrm{TM}_k$.
- **(6)** Presence link: the task interval $x_i$ is present if and only if at least one of its logical instances $y_{pi}$ is present.
- **(7)** Synchronization link: if a logical instance $y_{pi}$ is present, it must be synchronized (same start/end) with the corresponding task interval $x_i$.

**Variables:**
- **(8a)** $x_i$: optional interval variable for task $i$ (size $PT_i$).
- **(8b)** $y_{pi}$: optional interval variables representing child tasks in AND/OR structures (size $PT_i$).
- **(8c)** $x_p$: optional interval variable for an abstract parent node $p$.
- **(8d)** $\mathrm{seq}_k$: sequence variable for machine $k$ ordering the intervals $\{x_i\mid h_i=k\}$.

#### Symbols and Notation

| Symbol / Function | Meaning | docplex.cp reference |
|---|---|---|
| $N$ | Number of tasks (indexed $i\in[1..N]$) | — |
| $M$ | Number of machines (unary renewable resources indexed $k\in[1..M]$) | — |
| $PT_i$ | Duration (processing time) of task $i$ | — |
| $h_i$ | Fixed machine assignment of task $i$ ($h_i\in[1..M]$) | — |
| $P_{\text{AND}},\,P_{\text{OR}}$ | Sets of AND / OR parent nodes in the process structure | — |
| $A[p]$ | Children of parent node $p$ (for AND/OR logic) | — |
| $L=(\ell_{ij})$ | Time-lag matrix enforcing $s_i+\ell_{ij}\le s_j$ | — |
| $S^{(k)}_{ij}$ | Sequence-dependent setup time from task $i$ to $j$ on machine $k$ | — |
| $\mathrm{TM}_k=[S^{(k)}_{ij}]$ | Transition (setup) matrix for machine $k$ used in sequencing | [transition_matrix](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.transition_matrix) |
| $x_i$ | Optional interval variable for task $i$ (size $PT_i$) | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $x_p$ | Optional interval variable for abstract parent $p$ | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $y_{pi}$ | Optional child interval variable for parent $p$ and child $i\in A[p]$ | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $\mathrm{seq}_k$ | Sequence variable representing the order of tasks assigned to machine $k$ | [sequence_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.sequence_var) |
| $\mathrm{span}(x_p,[y_{pi}])$ | AND node: $x_p$ spans all present child intervals $y_{pi}$ | [span](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.span) |
| $\mathrm{alternative}(x_p,[y_{pi}])$ | OR node: select exactly one child synchronized with $x_p$ | [alternative](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.alternative) |
| $\mathrm{presenceOf}(x)$ | Returns 1 if interval $x$ is present | [presence_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) |
| $\mathrm{startOf}(x)$ | Start time of interval $x$ | [start_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.start_of) |
| $\mathrm{endOf}(x_i)$ | End time of interval $x_i$ | [end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\mathrm{startBeforeStart}(x_i,x_j,\ell_{ij})$ | Time-lag constraint enforcing $s_i+\ell_{ij}\le s_j$ | [start_before_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.start_before_start) |
| $\mathrm{noOverlap}(\mathrm{seq}_k,\mathrm{TM}_k)$ | Enforces unary capacity on machine $k$ with setup times $S^{(k)}_{ij}$ | [no_overlap](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.no_overlap) |
| $\max_i\,\mathrm{endOf}(x_i)$ | Makespan in sinkless formulation | [end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min C_{\max}$ | Objective: minimize makespan $C_{\max}$ | [minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |